In [4]:
import requests
import json

import numpy as np
from tqdm import tqdm
import pandas as pd
from git import Repo, Commit

In [3]:
r = requests.get("https://raw.githubusercontent.com/nluedtke/linux_kernel_cves/master/data/kernel_cves.json")
with open("data/kernel_cves.json", "w") as f:
    f.write(r.text)

In [10]:
with open("data/kernel_cves.json", "r") as f:
    kernel_cves = json.load(f)

In [45]:
kernel_cves_2015 = [cve for cve in kernel_cves if int(cve.split("-")[1]) >= 2015]
cves_with_fixes = [cve for cve in kernel_cves_2015 if kernel_cves[cve].get("fixes") and len(kernel_cves[cve]["fixes"]) == 40]

In [ ]:
linux = Repo("data/repos/linux")

In [ ]:
header = True
for cve in tqdm(cves_with_fixes):
    try:
        data = kernel_cves[cve]
        sha = data["fixes"]
        commit = linux.commit(sha)
        output = {
            "commit_msg": commit.message,
            "sha": sha,
            "remote_url": commit.repo.remotes.origin.url,
            "date": commit.committed_datetime,
            "labels": 1,
        }
        df = pd.DataFrame([output])
        df.to_csv(f"data/linux_cves.csv", mode="a", header=header, index=False)
        header = False
    except Exception as e:
        print(e)
        continue

In [ ]:
commits = list(linux.iter_commits(since="2015-01-01", no_merges=True))

In [ ]:
sample_size = 10_000
sample: list[Commit] = np.random.choice(commits, sample_size, replace=False)

In [ ]:
header = True
for commit in tqdm(sample):
    try:
        output = {
            "commit_msg": commit.message,
            "sha": commit.hexsha,
            "remote_url": commit.repo.remotes.origin.url,
            "date": commit.committed_datetime,
            "labels": -1,
        }
        df = pd.DataFrame([output])
        df.to_csv(f"data/linux.csv", mode="a", header=header, index=False)
        header = False
    except Exception as e:
        print(e)
        continue